# CATBoost


---> Ordering Boosting (Butun datatyi sirayla kulanmak yerine aradan secme yaparak siralayip, tahmin yapar) diye bir method kullanir.

---> Overfittinge karsi dayaniklidir.

---> Overfitting; ML modelimizin train ettigimiz data setinde cok iyi operformans gosterip, daha oncden gormedigi test datasinda kotu perfornans gostermesidir. Model train veri setini ezberliyor, ama testte basarisiz oluyor.
Bizim ML dekki amacimiz veri setini ezberlemek degil, GENELLEMEKtir.

---> CatBoost kategorik degiskenlerde daha iyi performans sergiler.

---> Burada data setinde scaling veya encodig yapmaya gerek yk. Data setini, features ve targetteki dkategorik verilele hic ugrasmadan direk verebiliyoruz CatBoost'a.

In [17]:
# !nvidia-smi # GPU calisiyor mu bakiyoruz

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
!pip install  catboost


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [38]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report



In [39]:
# Adult-Census-Income veri seti
data = pd.read_csv("https://raw.githubusercontent.com/pooja2512/Adult-Census-Income/master/adult.csv")

In [40]:
data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education.num   32561 non-null  int64 
 5   marital.status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital.gain    32561 non-null  int64 
 11  capital.loss    32561 non-null  int64 
 12  hours.per.week  32561 non-null  int64 
 13  native.country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [23]:
(data["workclass"] =="?").any()
# Bu seride en az 1 tane ? varsa Ture dondurur


True

In [24]:
columns_with_qmark = [col for col in data.columns if  (data[col] =="?").any()]
columns_with_qmark

['workclass', 'occupation', 'native.country']

In [25]:
for col in columns_with_qmark:
  data[col] = data[col].replace("?", "Unknown")

data.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,Unknown,77053,HS-grad,9,Widowed,Unknown,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,Unknown,186061,Some-college,10,Widowed,Unknown,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [42]:
data["income"].value_counts(normalize = True)*100

<=50K    75.919044
>50K     24.080956
Name: income, dtype: float64

In [27]:
# veris seti dengesiz

In [43]:
X, y =data.drop(["income"], axis = 1), data["income"]

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.2,
                                                    random_state = 34589,
                                                    stratify = y
                                                    )

In [45]:
X_test

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
30562,29,Private,105694,HS-grad,9,Married-civ-spouse,Handlers-cleaners,Husband,White,Male,0,0,42,United-States
29604,38,Private,207202,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,48,United-States
6785,48,Self-emp-inc,26145,10th,6,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,80,United-States
12879,37,Self-emp-not-inc,301568,HS-grad,9,Married-civ-spouse,Prof-specialty,Wife,White,Female,0,0,35,United-States
19045,41,Private,206948,Bachelors,13,Married-civ-spouse,Sales,Husband,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22075,52,Private,177366,HS-grad,9,Separated,Other-service,Other-relative,White,Female,0,0,20,United-States
13985,29,State-gov,409201,Some-college,10,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States
7855,30,Private,98733,Some-college,10,Divorced,Machine-op-inspct,Unmarried,White,Female,0,0,20,United-States
6074,46,Private,27669,HS-grad,9,Never-married,Other-service,Not-in-family,White,Female,0,0,28,United-States


In [46]:
X_train

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
6647,25,Private,216010,Assoc-acdm,12,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,40,United-States
29774,27,Self-emp-not-inc,357283,HS-grad,9,Never-married,Sales,Not-in-family,Black,Male,0,0,40,United-States
7340,25,Local-gov,44216,HS-grad,9,Divorced,Adm-clerical,Not-in-family,Amer-Indian-Eskimo,Female,0,0,35,United-States
9052,38,Local-gov,134444,Some-college,10,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,72,United-States
24146,37,Private,83893,Assoc-voc,11,Married-civ-spouse,Adm-clerical,Husband,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24449,51,Federal-gov,140516,Masters,14,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States
27376,60,Self-emp-not-inc,21101,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,Amer-Indian-Eskimo,Male,0,0,50,United-States
12036,34,Self-emp-not-inc,204375,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Female,0,0,60,United-States
31285,38,Private,54317,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,60,United-States


In [32]:
# Data setinde categorikleri degistirmeden, scaling yapmadan oldugu gibi CatBoost'a gidebiliriz.

## Modeli Kuralim

In [47]:
# kategorik sutunlari belirleyelim;

categoricial_features = list(X_train.select_dtypes(include = "object").columns)
categoricial_features

['workclass',
 'education',
 'marital.status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native.country']

In [48]:
train_pool = Pool(X_train, y_train, cat_features = categoricial_features)
test_pool = Pool(X_test, y_test, cat_features = categoricial_features)
# Pool veri havuzu oluyor. Data seti buyuk oldugunda dta setini saklama kutuphanesi bu. Ram'a butun datatyi birden gondermiyor.
# Ram isledikce yukluyor datayi ram'a. Bilgisayarin performansini artirir, zaman kazandirir bize.
# ML - DL kutuphanelerinde Pool gibi fonksiyonlara Data API denir. Bunlar training isleminin daha hizli gerceklestirilesini saglar.
# Artik bizim elimizde bir veri havuzu var. Buradan 1 tane veri aldigimizda, anyi anda hem feature'i hem de targer'i alacagiz.


In [49]:
# Modeli kuralim;
model = CatBoostClassifier(n_estimators = 300, # Biz 300 dedik ama en iyi sonucu aldiginda "early stopping: Erken durdurma" islemi yapar.
                           task_type = "GPU",  # Egitim taskinin GPU uzerinde yappilmasini istedik
                           )



In [50]:
param_dist = {"learning_rate": [0.03, 0.1, 0.3],
             "depth": [3, 5, 7],
             "l2_leaf_reg": [1, 3, 5]}
 # "depth": [3, 5, 7  Buraya tek sayilari vermek daha iyi sonuc veririr
# "l2_leaf_reg" deki sayilar arttikca regularizasyon (overfittinge karsi gelistirilmis bir yontemdir) etkisi o kadar artar.
# Bu degeri cok artirirsak mo0del ogrenemz (Underfitting  olur).

## RandomSearch
Az yukarida verdigimiz parametreleri randomize secerek  optimim hyper parametreleri bulacak.

In [56]:
""" 
Colab GPU'da calisacaksan bu kodu kullan 
search_resuts = model.randomized_search(param_dist, train_pool, cv=3, n_iter = 10, partition_random_seed = 54574, verbose = 1 ) 
search_resuts
# Uzun (20-30 dk) surer. CPU da calisirsak bu sure cok daha fazla olur. """

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/cuda/cuda_lib/cuda_manager.cpp:201: Condition violated: `State == nullptr'

In [57]:
# Jupyter'de calisacaksan bunu kullan; 
# CatBoost modelini oluşturun
model = CatBoostClassifier(iterations=500, loss_function='Logloss')

# Randomized Search'ü gerçekleştirin
search_results = model.randomized_search(
    param_dist,
    train_pool,
    cv=3,
    n_iter=10,
    partition_random_seed=54574,
    verbose=1
)

# Sonuçları görüntüleyin
search_results

0:	learn: 0.6654151	test: 0.6654817	best: 0.6654817 (0)	total: 177ms	remaining: 1m 28s
1:	learn: 0.6398127	test: 0.6401560	best: 0.6401560 (1)	total: 194ms	remaining: 48.3s
2:	learn: 0.6169336	test: 0.6177092	best: 0.6177092 (2)	total: 211ms	remaining: 35s
3:	learn: 0.5960707	test: 0.5974875	best: 0.5974875 (3)	total: 227ms	remaining: 28.1s
4:	learn: 0.5770676	test: 0.5789516	best: 0.5789516 (4)	total: 245ms	remaining: 24.3s
5:	learn: 0.5594101	test: 0.5613749	best: 0.5613749 (5)	total: 261ms	remaining: 21.5s
6:	learn: 0.5438627	test: 0.5461703	best: 0.5461703 (6)	total: 279ms	remaining: 19.7s
7:	learn: 0.5297103	test: 0.5323476	best: 0.5323476 (7)	total: 294ms	remaining: 18.1s
8:	learn: 0.5167131	test: 0.5196070	best: 0.5196070 (8)	total: 309ms	remaining: 16.9s
9:	learn: 0.5042266	test: 0.5067223	best: 0.5067223 (9)	total: 325ms	remaining: 15.9s
10:	learn: 0.4929279	test: 0.4956905	best: 0.4956905 (10)	total: 341ms	remaining: 15.2s
11:	learn: 0.4817549	test: 0.4842508	best: 0.4842508 

104:	learn: 0.3183230	test: 0.3221185	best: 0.3221185 (104)	total: 2.09s	remaining: 7.88s
105:	learn: 0.3179680	test: 0.3217794	best: 0.3217794 (105)	total: 2.12s	remaining: 7.87s
106:	learn: 0.3177589	test: 0.3215477	best: 0.3215477 (106)	total: 2.13s	remaining: 7.84s
107:	learn: 0.3176045	test: 0.3213543	best: 0.3213543 (107)	total: 2.15s	remaining: 7.81s
108:	learn: 0.3173479	test: 0.3211142	best: 0.3211142 (108)	total: 2.17s	remaining: 7.78s
109:	learn: 0.3169892	test: 0.3207990	best: 0.3207990 (109)	total: 2.19s	remaining: 7.75s
110:	learn: 0.3167337	test: 0.3205702	best: 0.3205702 (110)	total: 2.21s	remaining: 7.74s
111:	learn: 0.3165399	test: 0.3204038	best: 0.3204038 (111)	total: 2.22s	remaining: 7.71s
112:	learn: 0.3162502	test: 0.3200891	best: 0.3200891 (112)	total: 2.24s	remaining: 7.68s
113:	learn: 0.3160325	test: 0.3198774	best: 0.3198774 (113)	total: 2.26s	remaining: 7.66s
114:	learn: 0.3157172	test: 0.3195893	best: 0.3195893 (114)	total: 2.28s	remaining: 7.64s
115:	learn

196:	learn: 0.3029402	test: 0.3082268	best: 0.3082268 (196)	total: 3.82s	remaining: 5.88s
197:	learn: 0.3029100	test: 0.3081676	best: 0.3081676 (197)	total: 3.84s	remaining: 5.86s
198:	learn: 0.3027792	test: 0.3081439	best: 0.3081439 (198)	total: 3.86s	remaining: 5.83s
199:	learn: 0.3027267	test: 0.3080885	best: 0.3080885 (199)	total: 3.88s	remaining: 5.82s
200:	learn: 0.3026635	test: 0.3080413	best: 0.3080413 (200)	total: 3.89s	remaining: 5.79s
201:	learn: 0.3025433	test: 0.3079352	best: 0.3079352 (201)	total: 3.91s	remaining: 5.77s
202:	learn: 0.3023797	test: 0.3078016	best: 0.3078016 (202)	total: 3.93s	remaining: 5.75s
203:	learn: 0.3023093	test: 0.3077148	best: 0.3077148 (203)	total: 3.95s	remaining: 5.73s
204:	learn: 0.3021861	test: 0.3076224	best: 0.3076224 (204)	total: 3.96s	remaining: 5.71s
205:	learn: 0.3019567	test: 0.3074321	best: 0.3074321 (205)	total: 3.98s	remaining: 5.68s
206:	learn: 0.3018412	test: 0.3073432	best: 0.3073432 (206)	total: 4s	remaining: 5.66s
207:	learn: 0

293:	learn: 0.2939458	test: 0.3012117	best: 0.3012117 (293)	total: 5.52s	remaining: 3.87s
294:	learn: 0.2936548	test: 0.3009565	best: 0.3009565 (294)	total: 5.54s	remaining: 3.85s
295:	learn: 0.2936112	test: 0.3009548	best: 0.3009548 (295)	total: 5.56s	remaining: 3.83s
296:	learn: 0.2934673	test: 0.3008341	best: 0.3008341 (296)	total: 5.58s	remaining: 3.81s
297:	learn: 0.2934149	test: 0.3007936	best: 0.3007936 (297)	total: 5.59s	remaining: 3.79s
298:	learn: 0.2933874	test: 0.3007949	best: 0.3007936 (297)	total: 5.61s	remaining: 3.77s
299:	learn: 0.2932752	test: 0.3007076	best: 0.3007076 (299)	total: 5.63s	remaining: 3.75s
300:	learn: 0.2932421	test: 0.3006774	best: 0.3006774 (300)	total: 5.64s	remaining: 3.73s
301:	learn: 0.2932241	test: 0.3006374	best: 0.3006374 (301)	total: 5.66s	remaining: 3.71s
302:	learn: 0.2931833	test: 0.3006117	best: 0.3006117 (302)	total: 5.68s	remaining: 3.69s
303:	learn: 0.2931693	test: 0.3005851	best: 0.3005851 (303)	total: 5.7s	remaining: 3.67s
304:	learn:

386:	learn: 0.2871213	test: 0.2954095	best: 0.2954095 (386)	total: 7.22s	remaining: 2.11s
387:	learn: 0.2870819	test: 0.2954056	best: 0.2954056 (387)	total: 7.25s	remaining: 2.09s
388:	learn: 0.2870303	test: 0.2953737	best: 0.2953737 (388)	total: 7.27s	remaining: 2.08s
389:	learn: 0.2870180	test: 0.2953597	best: 0.2953597 (389)	total: 7.29s	remaining: 2.06s
390:	learn: 0.2869947	test: 0.2953644	best: 0.2953597 (389)	total: 7.33s	remaining: 2.04s
391:	learn: 0.2869514	test: 0.2953911	best: 0.2953597 (389)	total: 7.35s	remaining: 2.02s
392:	learn: 0.2869385	test: 0.2953865	best: 0.2953597 (389)	total: 7.37s	remaining: 2.01s
393:	learn: 0.2868851	test: 0.2953647	best: 0.2953597 (389)	total: 7.39s	remaining: 1.99s
394:	learn: 0.2868147	test: 0.2952743	best: 0.2952743 (394)	total: 7.41s	remaining: 1.97s
395:	learn: 0.2867081	test: 0.2952397	best: 0.2952397 (395)	total: 7.43s	remaining: 1.95s
396:	learn: 0.2866209	test: 0.2951619	best: 0.2951619 (396)	total: 7.45s	remaining: 1.93s
397:	learn

...

150:	learn: 0.2637094	test: 0.2883959	best: 0.2883673 (148)	total: 4.3s	remaining: 9.94s
151:	learn: 0.2636547	test: 0.2883858	best: 0.2883673 (148)	total: 4.33s	remaining: 9.91s
152:	learn: 0.2636039	test: 0.2883913	best: 0.2883673 (148)	total: 4.35s	remaining: 9.87s
153:	learn: 0.2635320	test: 0.2883470	best: 0.2883470 (153)	total: 4.38s	remaining: 9.84s
154:	learn: 0.2634668	test: 0.2883434	best: 0.2883434 (154)	total: 4.41s	remaining: 9.81s
155:	learn: 0.2633985	test: 0.2883632	best: 0.2883434 (154)	total: 4.43s	remaining: 9.78s
156:	learn: 0.2632096	test: 0.2883026	best: 0.2883026 (156)	total: 4.46s	remaining: 9.75s
157:	learn: 0.2631231	test: 0.2882896	best: 0.2882896 (157)	total: 4.49s	remaining: 9.72s
158:	learn: 0.2629049	test: 0.2882426	best: 0.2882426 (158)	total: 4.52s	remaining: 9.69s
159:	learn: 0.2628373	test: 0.2882306	best: 0.2882306 (159)	total: 4.54s	remaining: 9.66s
160:	learn: 0.2624968	test: 0.2880588	best: 0.2880588 (160)	total: 4.57s	remaining: 9.63s
161:	learn:

246:	learn: 0.2532698	test: 0.2868350	best: 0.2867068 (228)	total: 7.17s	remaining: 7.34s
247:	learn: 0.2530656	test: 0.2866549	best: 0.2866549 (247)	total: 7.2s	remaining: 7.32s
248:	learn: 0.2529160	test: 0.2866035	best: 0.2866035 (248)	total: 7.23s	remaining: 7.29s
249:	learn: 0.2528808	test: 0.2865645	best: 0.2865645 (249)	total: 7.25s	remaining: 7.25s
250:	learn: 0.2528216	test: 0.2866189	best: 0.2865645 (249)	total: 7.28s	remaining: 7.22s
251:	learn: 0.2527395	test: 0.2866296	best: 0.2865645 (249)	total: 7.31s	remaining: 7.19s
252:	learn: 0.2526479	test: 0.2865956	best: 0.2865645 (249)	total: 7.34s	remaining: 7.16s
253:	learn: 0.2524155	test: 0.2864922	best: 0.2864922 (253)	total: 7.37s	remaining: 7.14s
254:	learn: 0.2523388	test: 0.2864945	best: 0.2864922 (253)	total: 7.4s	remaining: 7.11s
255:	learn: 0.2522420	test: 0.2864472	best: 0.2864472 (255)	total: 7.42s	remaining: 7.08s
256:	learn: 0.2521647	test: 0.2864625	best: 0.2864472 (255)	total: 7.45s	remaining: 7.05s
257:	learn: 

342:	learn: 0.2442146	test: 0.2846311	best: 0.2846035 (341)	total: 10s	remaining: 4.58s
343:	learn: 0.2441601	test: 0.2846323	best: 0.2846035 (341)	total: 10s	remaining: 4.55s
344:	learn: 0.2441106	test: 0.2846197	best: 0.2846035 (341)	total: 10.1s	remaining: 4.52s
345:	learn: 0.2440352	test: 0.2846268	best: 0.2846035 (341)	total: 10.1s	remaining: 4.49s
346:	learn: 0.2438882	test: 0.2845893	best: 0.2845893 (346)	total: 10.1s	remaining: 4.46s
347:	learn: 0.2438413	test: 0.2845925	best: 0.2845893 (346)	total: 10.2s	remaining: 4.43s
348:	learn: 0.2437690	test: 0.2846019	best: 0.2845893 (346)	total: 10.2s	remaining: 4.41s
349:	learn: 0.2434947	test: 0.2842498	best: 0.2842498 (349)	total: 10.2s	remaining: 4.38s
350:	learn: 0.2434635	test: 0.2842783	best: 0.2842498 (349)	total: 10.2s	remaining: 4.35s
351:	learn: 0.2434332	test: 0.2842725	best: 0.2842498 (349)	total: 10.3s	remaining: 4.32s
352:	learn: 0.2434065	test: 0.2842795	best: 0.2842498 (349)	total: 10.3s	remaining: 4.29s
353:	learn: 0.

434:	learn: 0.2372893	test: 0.2843093	best: 0.2839219 (391)	total: 12.7s	remaining: 1.9s
435:	learn: 0.2372327	test: 0.2843113	best: 0.2839219 (391)	total: 12.7s	remaining: 1.87s
436:	learn: 0.2371894	test: 0.2843393	best: 0.2839219 (391)	total: 12.8s	remaining: 1.84s
437:	learn: 0.2370848	test: 0.2843675	best: 0.2839219 (391)	total: 12.8s	remaining: 1.81s
438:	learn: 0.2370528	test: 0.2843858	best: 0.2839219 (391)	total: 12.8s	remaining: 1.78s
439:	learn: 0.2370037	test: 0.2843807	best: 0.2839219 (391)	total: 12.9s	remaining: 1.75s
440:	learn: 0.2369761	test: 0.2843745	best: 0.2839219 (391)	total: 12.9s	remaining: 1.73s
441:	learn: 0.2369099	test: 0.2843960	best: 0.2839219 (391)	total: 12.9s	remaining: 1.7s
442:	learn: 0.2367282	test: 0.2842810	best: 0.2839219 (391)	total: 13s	remaining: 1.67s
443:	learn: 0.2366905	test: 0.2842564	best: 0.2839219 (391)	total: 13s	remaining: 1.64s
444:	learn: 0.2366210	test: 0.2842588	best: 0.2839219 (391)	total: 13s	remaining: 1.61s
445:	learn: 0.2365

{'params': {'depth': 5, 'l2_leaf_reg': 1, 'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
     

In [59]:
search_results["params"]

{'depth': 5, 'l2_leaf_reg': 1, 'learning_rate': 0.1}

In [60]:
# Modeli best parametrelerle yeniden kuralim.

RS_model = CatBoostClassifier(depth= 5, l2_leaf_reg = 1, learning_rate = 0.1)
RS_model.fit(train_pool)

0:	learn: 0.5968926	total: 64.4ms	remaining: 1m 4s
1:	learn: 0.5313621	total: 131ms	remaining: 1m 5s
2:	learn: 0.4847612	total: 174ms	remaining: 57.9s
3:	learn: 0.4490796	total: 213ms	remaining: 53s
4:	learn: 0.4184810	total: 250ms	remaining: 49.7s
5:	learn: 0.3987564	total: 297ms	remaining: 49.2s
6:	learn: 0.3798027	total: 338ms	remaining: 47.9s
7:	learn: 0.3672004	total: 385ms	remaining: 47.7s
8:	learn: 0.3582260	total: 415ms	remaining: 45.7s
9:	learn: 0.3508307	total: 445ms	remaining: 44.1s
10:	learn: 0.3454794	total: 474ms	remaining: 42.6s
11:	learn: 0.3406366	total: 506ms	remaining: 41.7s
12:	learn: 0.3372846	total: 554ms	remaining: 42.1s
13:	learn: 0.3327152	total: 585ms	remaining: 41.2s
14:	learn: 0.3289800	total: 618ms	remaining: 40.6s
15:	learn: 0.3253697	total: 649ms	remaining: 39.9s
16:	learn: 0.3227830	total: 680ms	remaining: 39.3s
17:	learn: 0.3197550	total: 710ms	remaining: 38.7s
18:	learn: 0.3186170	total: 738ms	remaining: 38.1s
19:	learn: 0.3165734	total: 771ms	remainin

162:	learn: 0.2671239	total: 5.38s	remaining: 27.6s
163:	learn: 0.2670917	total: 5.41s	remaining: 27.6s
164:	learn: 0.2670051	total: 5.45s	remaining: 27.6s
165:	learn: 0.2668967	total: 5.48s	remaining: 27.5s
166:	learn: 0.2667841	total: 5.51s	remaining: 27.5s
167:	learn: 0.2666631	total: 5.54s	remaining: 27.4s
168:	learn: 0.2665549	total: 5.57s	remaining: 27.4s
169:	learn: 0.2664520	total: 5.6s	remaining: 27.3s
170:	learn: 0.2662904	total: 5.63s	remaining: 27.3s
171:	learn: 0.2662588	total: 5.66s	remaining: 27.3s
172:	learn: 0.2661673	total: 5.69s	remaining: 27.2s
173:	learn: 0.2660059	total: 5.72s	remaining: 27.2s
174:	learn: 0.2658818	total: 5.75s	remaining: 27.1s
175:	learn: 0.2657941	total: 5.79s	remaining: 27.1s
176:	learn: 0.2656904	total: 5.82s	remaining: 27s
177:	learn: 0.2656270	total: 5.85s	remaining: 27s
178:	learn: 0.2653980	total: 5.88s	remaining: 27s
179:	learn: 0.2652959	total: 5.91s	remaining: 26.9s
180:	learn: 0.2651088	total: 5.94s	remaining: 26.9s
181:	learn: 0.26504

321:	learn: 0.2542622	total: 10.5s	remaining: 22.1s
322:	learn: 0.2542185	total: 10.6s	remaining: 22.1s
323:	learn: 0.2541630	total: 10.6s	remaining: 22.1s
324:	learn: 0.2541345	total: 10.6s	remaining: 22.1s
325:	learn: 0.2540357	total: 10.7s	remaining: 22s
326:	learn: 0.2539457	total: 10.7s	remaining: 22s
327:	learn: 0.2538332	total: 10.7s	remaining: 22s
328:	learn: 0.2537841	total: 10.8s	remaining: 21.9s
329:	learn: 0.2537419	total: 10.8s	remaining: 21.9s
330:	learn: 0.2537076	total: 10.8s	remaining: 21.9s
331:	learn: 0.2536694	total: 10.8s	remaining: 21.8s
332:	learn: 0.2536360	total: 10.9s	remaining: 21.8s
333:	learn: 0.2535426	total: 10.9s	remaining: 21.8s
334:	learn: 0.2535043	total: 10.9s	remaining: 21.7s
335:	learn: 0.2534572	total: 11s	remaining: 21.7s
336:	learn: 0.2533998	total: 11s	remaining: 21.7s
337:	learn: 0.2533462	total: 11s	remaining: 21.6s
338:	learn: 0.2533192	total: 11.1s	remaining: 21.6s
339:	learn: 0.2532624	total: 11.1s	remaining: 21.6s
340:	learn: 0.2531234	to

485:	learn: 0.2441766	total: 16s	remaining: 16.9s
486:	learn: 0.2441378	total: 16s	remaining: 16.9s
487:	learn: 0.2440754	total: 16s	remaining: 16.8s
488:	learn: 0.2440401	total: 16.1s	remaining: 16.8s
489:	learn: 0.2439730	total: 16.1s	remaining: 16.8s
490:	learn: 0.2439132	total: 16.1s	remaining: 16.7s
491:	learn: 0.2438474	total: 16.2s	remaining: 16.7s
492:	learn: 0.2437867	total: 16.2s	remaining: 16.7s
493:	learn: 0.2437814	total: 16.2s	remaining: 16.6s
494:	learn: 0.2437062	total: 16.3s	remaining: 16.6s
495:	learn: 0.2436451	total: 16.3s	remaining: 16.6s
496:	learn: 0.2436233	total: 16.3s	remaining: 16.5s
497:	learn: 0.2435228	total: 16.4s	remaining: 16.5s
498:	learn: 0.2434692	total: 16.4s	remaining: 16.5s
499:	learn: 0.2434443	total: 16.4s	remaining: 16.4s
500:	learn: 0.2433944	total: 16.5s	remaining: 16.4s
501:	learn: 0.2433030	total: 16.5s	remaining: 16.4s
502:	learn: 0.2432642	total: 16.5s	remaining: 16.3s
503:	learn: 0.2431976	total: 16.6s	remaining: 16.3s
504:	learn: 0.2431

644:	learn: 0.2364709	total: 21.4s	remaining: 11.8s
645:	learn: 0.2364672	total: 21.5s	remaining: 11.8s
646:	learn: 0.2364246	total: 21.5s	remaining: 11.7s
647:	learn: 0.2363981	total: 21.5s	remaining: 11.7s
648:	learn: 0.2363255	total: 21.6s	remaining: 11.7s
649:	learn: 0.2362814	total: 21.6s	remaining: 11.6s
650:	learn: 0.2362675	total: 21.6s	remaining: 11.6s
651:	learn: 0.2362425	total: 21.7s	remaining: 11.6s
652:	learn: 0.2362000	total: 21.7s	remaining: 11.5s
653:	learn: 0.2361632	total: 21.7s	remaining: 11.5s
654:	learn: 0.2361336	total: 21.7s	remaining: 11.5s
655:	learn: 0.2360928	total: 21.8s	remaining: 11.4s
656:	learn: 0.2360783	total: 21.8s	remaining: 11.4s
657:	learn: 0.2360426	total: 21.8s	remaining: 11.4s
658:	learn: 0.2360221	total: 21.9s	remaining: 11.3s
659:	learn: 0.2360183	total: 21.9s	remaining: 11.3s
660:	learn: 0.2359947	total: 21.9s	remaining: 11.3s
661:	learn: 0.2359822	total: 22s	remaining: 11.2s
662:	learn: 0.2359787	total: 22s	remaining: 11.2s
663:	learn: 0.23

808:	learn: 0.2294784	total: 27s	remaining: 6.38s
809:	learn: 0.2294459	total: 27.1s	remaining: 6.35s
810:	learn: 0.2294050	total: 27.1s	remaining: 6.33s
811:	learn: 0.2293507	total: 27.2s	remaining: 6.29s
812:	learn: 0.2293134	total: 27.2s	remaining: 6.26s
813:	learn: 0.2292605	total: 27.3s	remaining: 6.23s
814:	learn: 0.2292257	total: 27.3s	remaining: 6.2s
815:	learn: 0.2291915	total: 27.3s	remaining: 6.16s
816:	learn: 0.2291455	total: 27.4s	remaining: 6.13s
817:	learn: 0.2291159	total: 27.4s	remaining: 6.09s
818:	learn: 0.2291092	total: 27.4s	remaining: 6.06s
819:	learn: 0.2290946	total: 27.4s	remaining: 6.03s
820:	learn: 0.2290679	total: 27.5s	remaining: 5.99s
821:	learn: 0.2290431	total: 27.5s	remaining: 5.96s
822:	learn: 0.2290042	total: 27.5s	remaining: 5.92s
823:	learn: 0.2289533	total: 27.6s	remaining: 5.89s
824:	learn: 0.2289258	total: 27.6s	remaining: 5.86s
825:	learn: 0.2288777	total: 27.6s	remaining: 5.82s
826:	learn: 0.2288181	total: 27.7s	remaining: 5.79s
827:	learn: 0.2

971:	learn: 0.2225915	total: 32.4s	remaining: 932ms
972:	learn: 0.2225496	total: 32.4s	remaining: 899ms
973:	learn: 0.2225367	total: 32.4s	remaining: 866ms
974:	learn: 0.2225163	total: 32.5s	remaining: 832ms
975:	learn: 0.2224833	total: 32.5s	remaining: 799ms
976:	learn: 0.2224455	total: 32.5s	remaining: 766ms
977:	learn: 0.2223631	total: 32.6s	remaining: 732ms
978:	learn: 0.2223491	total: 32.6s	remaining: 699ms
979:	learn: 0.2223306	total: 32.6s	remaining: 666ms
980:	learn: 0.2222758	total: 32.7s	remaining: 632ms
981:	learn: 0.2222593	total: 32.7s	remaining: 599ms
982:	learn: 0.2222209	total: 32.7s	remaining: 566ms
983:	learn: 0.2221383	total: 32.7s	remaining: 533ms
984:	learn: 0.2220507	total: 32.8s	remaining: 499ms
985:	learn: 0.2220393	total: 32.8s	remaining: 466ms
986:	learn: 0.2220127	total: 32.8s	remaining: 433ms
987:	learn: 0.2219727	total: 32.9s	remaining: 399ms
988:	learn: 0.2219425	total: 32.9s	remaining: 366ms
989:	learn: 0.2219155	total: 32.9s	remaining: 333ms
990:	learn: 

In [61]:
y_train_pred = RS_model.predict(train_pool)
y_train_pred

array(['<=50K', '<=50K', '<=50K', ..., '>50K', '<=50K', '<=50K'],
      dtype=object)

In [62]:
y_pred = RS_model.predict(test_pool)
y_pred

array(['<=50K', '<=50K', '<=50K', ..., '<=50K', '<=50K', '<=50K'],
      dtype=object)

## Model Evaluation

In [63]:
print(classification_report(y_train, y_train_pred, digits = 3))

              precision    recall  f1-score   support

       <=50K      0.914     0.953     0.933     19775
        >50K      0.830     0.716     0.768      6273

    accuracy                          0.896     26048
   macro avg      0.872     0.835     0.851     26048
weighted avg      0.893     0.896     0.893     26048



In [64]:
print(classification_report(y_test, y_pred, digits = 3))

              precision    recall  f1-score   support

       <=50K      0.898     0.938     0.918      4945
        >50K      0.772     0.666     0.715      1568

    accuracy                          0.872      6513
   macro avg      0.835     0.802     0.816      6513
weighted avg      0.868     0.872     0.869      6513



**YORUM:** Modelin accuracy dageri trainde 0.89, testte 0.87 olup, model basarilidir.

In [ ]:
# Done